In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
spark = SparkSession.builder.appName("Explode and Join").getOrCreate()
commentary_data = [
    ("1", "Team A", ["Player 1", "Player 2", "Player 3", "Captain A"]),
    ("2", "Team B", ["Player 4", "Player 5", "Captain B", "Player 6"])
]

commentary_columns = ["match_id", "team", "playing11"]

commentary_df = spark.createDataFrame(commentary_data, commentary_columns)

commentary_df.show(truncate=False)



+--------+------+-----------------------------------------+
|match_id|team  |playing11                                |
+--------+------+-----------------------------------------+
|1       |Team A|[Player 1, Player 2, Player 3, Captain A]|
|2       |Team B|[Player 4, Player 5, Captain B, Player 6]|
+--------+------+-----------------------------------------+



In [0]:
player_href_data = [
    ("Player 1", "Batsman", "No"),
    ("Player 2", "Bowler", "No"),
    ("Player 3", "Allrounder", "No"),
    ("Captain A", "Batsman", "Yes"),
    ("Player 4", "Bowler", "No"),
    ("Player 5", "Allrounder", "No"),
    ("Captain B", "Batsman", "Yes"),
    ("Player 6", "Bowler", "No")
]

player_href_columns = ["player_name", "role", "is_captain"]

player_href_df = spark.createDataFrame(player_href_data, player_href_columns)

player_href_df.show(truncate=False)


+-----------+----------+----------+
|player_name|role      |is_captain|
+-----------+----------+----------+
|Player 1   |Batsman   |No        |
|Player 2   |Bowler    |No        |
|Player 3   |Allrounder|No        |
|Captain A  |Batsman   |Yes       |
|Player 4   |Bowler    |No        |
|Player 5   |Allrounder|No        |
|Captain B  |Batsman   |Yes       |
|Player 6   |Bowler    |No        |
+-----------+----------+----------+



In [0]:
exploded_df = commentary_df.select(
    "match_id",
    "team",
    explode("playing11").alias("player_name")
)

joined_df = exploded_df.join(
    player_href_df,
    on="player_name",
    how="left"
)

captains_df = joined_df.filter(joined_df.is_captain == "Yes")

captains_df.select("match_id", "team", "player_name", "role", "is_captain").show(truncate=False)


+--------+------+-----------+-------+----------+
|match_id|team  |player_name|role   |is_captain|
+--------+------+-----------+-------+----------+
|1       |Team A|Captain A  |Batsman|Yes       |
|2       |Team B|Captain B  |Batsman|Yes       |
+--------+------+-----------+-------+----------+

